# Purpose
- This file is a scratch-pad for sql queries

In [14]:
%load_ext sql
%sql duckdb:///../data/referral.db --alias referralsDB

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [15]:
%sql referralsDB

In [16]:
%%sql
CREATE OR REPLACE TABLE health_plan as 

SELECT DISTINCT * FROM read_csv_auto('../data/reference/health_plan.csv', header=True)

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
34


In [17]:
%%sql
CREATE OR REPLACE TABLE coc as

SELECT DISTINCT * FROM read_csv_auto('../data/reference/coc.csv', header=True)

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
818829


In [18]:
%%sql 
CREATE OR REPLACE TABLE team as 

SELECT DISTINCT * FROM read_csv_auto('../data/reference/user_list.csv', header=True)

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
266


In [19]:
%sql CREATE OR REPLACE TABLE hcpcs as SELECT DISTINCT * FROM read_csv_auto('../data/reference/cpt_hcpcs.csv', header=True)

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
16991


In [20]:
%sql CREATE OR REPLACE TABLE icd10cm as SELECT DISTINCT * FROM read_csv('../data/reference/icd10cm.csv', header=True, columns={'code_value': 'VARCHAR', 'code_long_description': 'VARCHAR'})

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
95387


In [21]:
%%sql 
CREATE OR REPLACE TABLE cln_referrals as 

SELECT DISTINCT * FROM read_parquet('../data/clean_referrals/*.parquet')

*  (referralsDB) duckdb:///../data/referral.db
(duckdb.IOException) IO Error: No files found that match the pattern "../data/clean_referrals/*.parquet"
[SQL: CREATE OR REPLACE TABLE cln_referrals as 

SELECT DISTINCT * FROM read_parquet('../data/clean_referrals/*.parquet')]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


- No Auth Required ->	Appointment Scheduled or Member to Schedule or Unable to Contact
- Approved	       ->   Appointment Scheduled or Member to Schedule or Unable to Contact
- Denied	       ->   Open
- Withdrawn	       ->   Open

In [22]:
%%sql 
CREATE OR REPLACE TABLE bi_referrals AS 

WITH main AS (

    SELECT DISTINCT rf.*
    , COALESCE(tm.Team, 'N/A') AS User_Team

    , UPPER("Last Name") || UPPER("First Name") AS patient_name

    , UPPER("Last Name") || UPPER("First Name") || "DOB" AS patient_id

    , patient_id || "Referring Provider NPI" || "Referral Date" || coalesce("Diagnosis", 'NA') || coalesce("Speciality", 'NA') AS Referral_keyid

    , CASE
        WHEN "Approval Status" IN ('No HP Auth Required', 'HP Approved', 'Approved (comments required)', 'Complete/no Auth# needed')  
            AND "Visit Status" IN ('Appointment Scheduled', 'Member to Schedule', 'Unable to Contact', 'Denied', 'Withdrawn')
            THEN 'Completed'
        WHEN "Approval Status" IN ('Denied- Per Insurance Plan', 'Denied- Per Medical Director Review')
            AND "Visit Status" = 'Open'
            THEN 'Completed'
        ELSE "Referral Status" END AS "Updated Status"

    , RIGHT("Procedure", 5) AS Proc_code
    , pc.code_short_description AS Proc_name 

    , FORMAT(
        '({}) {}-{}'
        , substring("Mobile Phone", 1, 3), substring("Mobile Phone", 3, 3), substring("Mobile Phone", 6, 4)
    ) AS "Fmt Mobile Phone"

    , FORMAT(
        '({}) {}-{}'
        , substring("Home Phone", 1, 3), substring("Home Phone", 3, 3), substring("Home Phone", 6, 4)
    ) AS "Fmt Home Phone"

    , REPLACE(REPLACE(REPLACE(REGEXP_REPLACE(UPPER("Health Plan"), '[(*)]', ' ', 'g'), '-', ' '), '  ', ' '), '  ', ' ') AS "Health Plan FMT"

    , regexp_replace(array_slice(Address, -9, -7), '[0-9]', '', 'g') AS STATE

    , array_slice(Address, -5, NULL) as ZIPCODE

    , icd.code_long_description AS "Diagnosis Description"

    FROM cln_referrals AS rf

    LEFT JOIN icd10cm AS icd
        ON trim(rf."Diagnosis") = trim(icd.code_value)

    LEFT JOIN hcpcs AS pc
        ON RIGHT("Procedure", 5) = pc.code_value

    LEFT JOIN team AS tm
        ON "User_FName" = tm.Fname
        AND "User_LName" = tm.Lname

    WHERE "Visit Status" IS NOT NULL
)

, max_updatedt AS (
    SELECT DISTINCT
        Referral_keyid
        , Update_DT
        , MAX(Update_DT) OVER (PARTITION BY Referral_keyid) as Last_UpdateDT

    FROM main 
)

, condense_dates AS (
    SELECT DISTINCT 
        m.* 
        , row_number() OVER (PARTITION BY patient_id) as patient_id_count

    FROM main AS m

    INNER JOIN max_updatedt as dt
        ON m.Update_DT = dt.Last_UpdateDT
        AND m.Referral_keyid = dt.Referral_keyid
)

, gatekeeper as (
    SELECT DISTINCT 
        cd.* 
        , hp.GATEKEEPER

    FROM condense_dates AS cd

    LEFT JOIN health_plan AS hp
        ON cd.STATE = hp.STATE
        AND cd.LOB = hp.LOB
)

SELECT DISTINCT * FROM gatekeeper

*  (referralsDB) duckdb:///../data/referral.db


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

Done.


Count
119911


In [23]:
%sql SELECT DISTINCT * FROM bi_referrals ORDER BY patient_id LIMIT 10

*  (referralsDB) duckdb:///../data/referral.db
Done.


C0,Center,Clinic,Referring Provider,Referring Provider NPI,Referral Date,Last Name,First Name,Sex,DOB,pat Status,Language,Address,Mobile Phone,Home Phone,PCP,Referred to Specialist,Specialist NPI,Referral Status,Network,Specialty,Diagnosis,Procedure,Approval Status,Visit Status,Referral Type,Patient Type,Health Plan,Plan ID,LOB,Auth No,Last Update,Update_DT,file_source,User_Fname,User_LName,User_Team,patient_name,patient_id,Referral_keyid,Updated Status,Proc_code,Proc_name,Fmt Mobile Phone,Fmt Home Phone,Health Plan FMT,STATE,ZIPCODE,Diagnosis Description,patient_id_count,GATEKEEPER
1062,Centrum Health - Texas,(H) S SAM HOUSTON PKY E,"Denna Foreman, APRN",1942804141,2023-03-27,A ABBOTT,ADONIS,M,1976-04-03,Active,English,"5638 Windsor Forest DrHouston, TX, 77088",7132948740,7132948740,"Foreman, Denna",Dr. Suniti D. Patel,None,Open,Out of network,Podiatry,B35.1,None,HP Approved,Pending Appt,External,Adult,AMBETTER SUPERIOR HEALTH- EXCHANGE,U9690656001,Exchange,REF490CC614F,"YOUNG, CASSADY",Mar 29 2023 3:53PM,../data/raw_referrals/excels/TX REFERRAL 3_1-3_31.xlsx,CASSADY,YOUNG,N/A,A ABBOTTADONIS,A ABBOTTADONIS1976-04-03,A ABBOTTADONIS1976-04-0319428041412023-03-27B35.1,Open,None,None,(713) 329-4874,(713) 329-4874,AMBETTER SUPERIOR HEALTH EXCHANGE,TX,77088,TINEA UNGUIUM,9,None
914,Centrum Health - Texas,(H) S SAM HOUSTON PKY E,"Denna Foreman, APRN",1942804141,2023-03-27,A ABBOTT,ADONIS,M,1976-04-03,Active,English,"5638 Windsor Forest DrHouston, TX, 77088",7132948740,7132948740,"Foreman, Denna",dr scott e. segal,None,Open,Out of network,Ophthalmology,H40.9,None,Complete/no Auth# needed,Open,External,Adult,AMBETTER SUPERIOR HEALTH- EXCHANGE,U9690656001,Exchange,REF99D1690E1,"CRUZ, KELIN",Mar 27 2023 4:20PM,../data/raw_referrals/excels/TX REFERRALS 3_1-3_29.xlsx,KELIN,CRUZ,N/A,A ABBOTTADONIS,A ABBOTTADONIS1976-04-03,A ABBOTTADONIS1976-04-0319428041412023-03-27H40.9,Open,None,None,(713) 329-4874,(713) 329-4874,AMBETTER SUPERIOR HEALTH EXCHANGE,TX,77088,UNSPECIFIED GLAUCOMA,17,None
1062,Centrum Health - Texas,(H) S SAM HOUSTON PKY E,"Denna Foreman, APRN",1942804141,2023-03-27,A ABBOTT,ADONIS,M,1976-04-03,Active,English,"5638 Windsor Forest DrHouston, TX, 77088",7132948740,7132948740,"Foreman, Denna",Dr. Suniti D. Patel,None,Open,Out of network,Podiatry,B35.1,None,HP Approved,Pending Appt,External,Adult,AMBETTER SUPERIOR HEALTH- EXCHANGE,U9690656001,Exchange,REF490CC614F,"YOUNG, CASSADY",Mar 29 2023 3:53PM,../data/raw_referrals/excels/TX REFERRAL 3_1-3_31.xlsx,CASSADY,YOUNG,N/A,A ABBOTTADONIS,A ABBOTTADONIS1976-04-03,A ABBOTTADONIS1976-04-0319428041412023-03-27B35.1,Open,None,None,(713) 329-4874,(713) 329-4874,AMBETTER SUPERIOR HEALTH EXCHANGE,TX,77088,TINEA UNGUIUM,14,None
1008,Centrum Health - Texas,(H) S SAM HOUSTON PKY E,"Denna Foreman, APRN",1942804141,2023-03-27,A ABBOTT,ADONIS,M,1976-04-03,Active,English,"5638 Windsor Forest DrHouston, TX, 77088",7132948740,7132948740,"Foreman, Denna",Dr. Suniti D. Patel,None,Open,Out of network,Podiatry,B35.1,None,HP Approved,Pending Appt,External,Adult,AMBETTER SUPERIOR HEALTH- EXCHANGE,U9690656001,Exchange,REF490CC614F,"YOUNG, CASSADY",Mar 29 2023 3:53PM,../data/raw_referrals/excels/TX REFERRALS 3_1-3_30.xlsx,CASSADY,YOUNG,N/A,A ABBOTTADONIS,A ABBOTTADONIS1976-04-03,A ABBOTTADONIS1976-04-0319428041412023-03-27B35.1,Open,None,None,(713) 329-4874,(713) 329-4874,AMBETTER SUPERIOR HEALTH EXCHANGE,TX,77088,TINEA UNGUIUM,2,None
841,Centrum Health - Texas,(H) S SAM HOUSTON PKY E,"Denna Foreman, APRN",1942804141,2023-03-27,A ABBOTT,ADONIS,M,1976-04-03,Active,English,"5638 Windsor Forest DrHouston, TX, 77088",7132948740,7132948740,"Foreman, Denna",dr scott e. segal,None,Open,Out of network,Ophthalmology,H40.9,None,Complete/no Auth# needed,Open,External,Adult,AMBETTER SUPERIOR HEALTH- EXCHANGE,U9690656001,Exchange,REF99D1690E1,"CRUZ, KELIN",Mar 27 2023 4:20PM,../data/raw_referrals/excels/TX REFERRALS 3_1-3_27.xlsx,KELIN,CRUZ,N/A,A ABBOTTADONIS,A ABBOTTADONIS1976-04-03,A ABBOTTADONIS1976-04-0319428041412